In [5]:
import yaml
import numpy as np
from scipy.optimize import least_squares
import pickle

initial_guess = [7, 30, 120]

with open('data.pkl', 'rb') as file:
    accumulated_positions = pickle.load(file)

def estimate_and_save_transform():
    # Example of processing; actual computation would depend on your application
    if len(accumulated_positions['gv01']) < 10:  # arbitrary number to ensure enough data
        return

    # Perform least squares optimization
    result = least_squares(
        error_function, initial_guess,
        args=(accumulated_positions['gv01'], accumulated_positions['gv02'])
    )

    # The optimized hidden variables
    # optimized_D, optimized_theta1, optimized_theta2 = result.x
    print(result.x)
        
def error_function(hidden_variables, detections_jeep, detections_wolf):
    D, theta1, theta2 = hidden_variables
    errors = []
    
    for (x_jeep, y_jeep), (x_wolf, y_wolf) in zip(detections_jeep, detections_wolf):
        
        det_jeep = np.array([x_jeep, y_jeep])
        det_wolf = np.array([x_wolf, y_wolf]).reshape((-1,1))
        
        # Move the Wolf detections into Jeep space using hidden variables
        rot_wolf = rotate(det_wolf, -theta2)
        translated_wolf = np.array([rot_wolf[0,0] - D, rot_wolf[1,0]]).reshape((-1,1))
        transformed_wolf = rotate(translated_wolf, theta1)
        
        # Calculate error in x and y
        error_x = det_jeep[0] - transformed_wolf[0,0]
        error_y = det_jeep[1] - transformed_wolf[1,0]
        
        errors.extend([error_x, error_y])
    
    return errors
            
def rotate(vec, degrees):

    R = np.array([[np.cos(np.deg2rad(degrees)), -np.sin(np.deg2rad(degrees))],
                    [np.sin(np.deg2rad(degrees)), np.cos(np.deg2rad(degrees))]])
    
    return R @ vec

estimate_and_save_transform()

[-4.06762204e-11  5.16428485e+01  5.16428485e+01]


In [7]:
print(accumulated_positions['gv01'])

[(12.876782659080162, -2.0671696698704722), (12.783466206632252, -1.833172625710466), (12.754195340331297, -1.7803480252815642), (12.744289892672473, -1.744118158572486), (12.435419762173751, -1.048381578943), (12.311159689398899, -0.9473752675750408), (12.285652295844452, -0.8968273589455446), (12.194505366663996, -0.7362599780885876), (12.17797254179999, -0.6942778357659524), (12.089362669440073, -0.4460418537061269), (12.065335245665876, -0.3845890959389212), (12.039637959722233, -0.3615339861639545), (11.93378484852788, -0.082450172132122), (11.91304188233902, -0.03777505686105688), (11.908015445605253, -0.02126992791505989), (11.780506021219248, 0.3276170195080933), (0.37329787688986693, -9.996439205851532), (0.41036943729793973, -9.97982844018735), (0.6175998092695261, -9.895974582209249), (0.6470818937731333, -9.885196047590712), (0.8320220048363144, -9.80950885978905), (0.8650997644379026, -9.797156511034284), (1.2236368523453767, -6.204974806083823), (1.1995370924701179, -6.12